<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/basics/data_rows.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/basics/data_rows.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Data rows

* Data rows are the assets that are being labeled. We currently support the following asset types:
    * Image
    * Text
    * Video
    * Geospatial / Tiled Imagery
    * Audio
    * Documents 
    * HTML 
    * DICOM 
    * Conversational
* A data row is a member of a dataset 
* A data row cannot exist without belonging to a dataset.
* Data rows are added to labeling tasks by first attaching them to datasets and then creating batches in projects

In [ ]:
!pip install labelbox -q

In [ ]:
import labelbox as lb
import uuid
import os

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = ""
client = lb.Client(api_key=API_KEY)

### Get data rows from projects

In [ ]:
# Pick a project with batches that have data rows with global keys
PROJECT_ID = "<PROJECT-ID>"
project = client.get_project(PROJECT_ID)
batches = next(project.batches())
print(batches)
# This is the same as
# -> dataset = client.get_dataset(dataset_id)

* Fetch data rows from project's batches

In [ ]:
data_rows = batches.export_data_rows()
data_row = next(data_rows)
print(data_row)

* Get labels from the data row

In [ ]:
print("Associated label(s)", list(data_row.labels()))
print("Global key", data_row.global_key)

* Get data row ids by using global keys

In [ ]:
global_key = "<ENTER GLOBAL KEY>"
task = client.get_data_row_ids_for_global_keys([global_key])
print(f"Data row id: {task['results']}")

### Create
* Create a single data row with and without metadata

In [ ]:
dataset = client.create_dataset(name="data_rows_demo_dataset")

# It is recommended that you add global keys to your data rows.
dataset.create_data_row(row_data="https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_0002.jpeg",
                        global_key=str(uuid.uuid4()))

# You can also upload metadata along with your data row
mdo = client.get_data_row_metadata_ontology()
dataset.create_data_row(row_data="https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_0003.jpeg",
                        global_key=str(uuid.uuid4()),
                        metadata_fields=[
                            lb.DataRowMetadataField(
                              schema_id=mdo.reserved_by_name["tag"].uid,  # specify the schema id
                              value="tag_string", # typed inputs
                            ),
                        ], 
)

* [Recommended] Bulk create data rows (This is much faster than creating individual data rows)

In [ ]:
# Create a dataset
dataset = client.create_dataset(name="data_rows_demo_dataset_2")

uploads = []
# Generate data rows
for i in range(1,9):
    uploads.append({
        "row_data":  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key": "TEST-ID-%id" % uuid.uuid1(),
        ## add metadata (optional)
        "metadata_fields": [
            lb.DataRowMetadataField(
                schema_id=mdo.reserved_by_name["tag"].uid,  # specify the schema id
                value="tag_string", # typed inputs
            ),
        ]
    })

task1 = dataset.create_data_rows(uploads)
task1.wait_till_done()
print("ERRORS: " , task1.errors)
print("RESULTS:" , task1.result)

* Create data rows with attachments

In [ ]:
task2 = dataset.create_data_rows([{
    "row_data": "https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_0009.jpeg",
    "global_key": str(uuid.uuid4()),
    "attachments": [
                {
                    "type": "IMAGE_OVERLAY",
                    "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/disease_attachment.jpeg"
                },
                {
                    "type": "RAW_TEXT",
                    "value": "IOWA, Zone 2232, June 2022 [Text string]"
                },
                {
                    "type": "TEXT_URL",
                    "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/text_attachment.txt"
                },
                {
                    "type": "IMAGE",
                    "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/disease_attachment.jpeg"
                },
                {
                    "type": "VIDEO",
                    "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/drone_video.mp4"
                },
                {
                    "type": "HTML",
                    "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/windy.html"
                }
            ]
    }])
print("ERRORS: " , task2.errors)
print("RESULTS:" , task2.result)

* Create data rows using data in your local path

In [ ]:
# Local paths
local_data_path = "/tmp/test_data_row.txt"
with open(local_data_path, 'w') as file:
    file.write("sample data")

task3 = dataset.create_data_rows([local_data_path])
print("ERRORS: " , task3.errors)
print("RESULTS:" , task3.result)

In [ ]:
# You can mix local files with urls when creating data rows
task4 = dataset.create_data_rows([{
    "row_data": "https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_0003.jpeg",
    "global_key": str(uuid.uuid4())
    }, {
    "row_data": local_data_path,
    "global_key": str(uuid.uuid4())
    }])
print("ERRORS: " , task4.errors)
print("RESULTS:" , task4.result)

### Update
Only two fields can be updated after a data row is created
1. Global keys 
2. Row data


In [ ]:
data_row = client.get_data_row("<data_row_id_to_update>")
new_id = str(uuid.uuid4())
data_row.update(global_key=new_id, row_data="https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_0005.jpeg")
print(data_row)

* Create a single attachemt on an existing data row

In [ ]:
# You can only create one attachment at the time. 
data_row.create_attachment(attachment_type="RAW_TEXT",
                           attachment_value="LABELERS WILL SEE THIS ")

### Delete

* Delete a single data row

In [ ]:
data_row = client.get_data_row("<data_row_id_to_delete>")
data_row.delete()

* Bulk delete data row objects

In [ ]:
# Bulk delete a list of data_rows ( limit: 4K data rows per call)
lb.DataRow.bulk_delete(list(dataset.data_rows()))